In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
# # RLE 디코딩 함수
# def rle_decode(mask_rle, shape):
#     starts, lengths = [np.asarray(x, dtype=int) for x in (mask_rle[::2], mask_rle[1::2])]
#     starts -= 1
#     ends = starts + lengths
#     img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
#     for lo, hi in zip(starts, ends):
#         img[lo:hi] = 255
#     return img.reshape(shape)

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
# transform = A.Compose(
#     [
#         A.Resize(224, 224),
#         A.Normalize(),
#         ToTensorV2()
#     ]
# )

# dataset = SatelliteDataset(csv_file='./train.csv', transform=transform)
# dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)

In [ ]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

In [ ]:
transform = A.Compose(
    [
        A.Resize(112, 112),
        A.Normalize(),
        ToTensorV2()
    ]
)

## **픽셀의 숫자가 특정 개수 이하면 -1**

In [ ]:
# 이미지 업로드해주세요
# 이미지 경로하고 검출한 마스크를 입력해주세요
# IMAGE_PATH = "/content/TEST_00121.png"
# MASK_RLE = "24248 4 24470 7 24694 7 24917 8 25141 8 25367 5 43988 2 44212 2 44436 3 44660 3 44884 3 46246 4 46470 4 46693 5 46917 5 47141 5 47366 3 49751 1 49975 1"


**컨투어 검출후 에러 삭제**

In [ ]:
# 컨투어 검출하기
def contoursDelet(mask):
  mask = np.array(mask)
  contours, hierarchy = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  # Threshold를 지정하세요.
  threshold = 50

  # Create an empty mask to store the pixels to be removed
  remove_mask = np.zeros_like(mask)

  # 각 컨투어를 순회하며 면적 계산
  for idx, contour in enumerate(contours):
    # 현재 컨투어가 최상위 컨투어이고(부모가 없고), 하위 컨투어(자식이)가 없으면 단일 구조
    if hierarchy[0][idx][3] == -1 and hierarchy[0][idx][2] == -1:
      area = cv2.contourArea(contour)
      #print(area)
      if area < threshold:
        cv2.drawContours(remove_mask, [contour], -1, 255, thickness=cv2.FILLED)

    # 현재 컨투어가 최상위 컨투어이고, 하위 컨투어(자식이)가 있으면 뚫린 구조
    elif hierarchy[0][idx][3] == -1 and hierarchy[0][idx][2] != -1:
      outer_area = cv2.contourArea(contour)
      inner_area = cv2.contourArea(contours[hierarchy[0][idx][2]])
      #print(outer_area - inner_area)
      if (outer_area - inner_area) < threshold:
        cv2.drawContours(remove_mask, [contour], -1, 255, thickness=cv2.FILLED)

  # Remove the small areas from the original mask
  final_mask = cv2.bitwise_and(mask, cv2.bitwise_not(remove_mask))

  return final_mask.astype(np.uint8)


In [ ]:
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device} device")

    transform = A.Compose(
        [
            A.Resize(224, 224),
            A.Normalize(),
            ToTensorV2()
        ]
    )

    test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform, infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

    model = UNet()
    model.load_state_dict(torch.load('./UNetB5.pth', map_location=torch.device('cpu')))
    model = model.to(device)

    # model = model.load_state_dict(torch.load("UNet.pth")).to(device)


with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        
        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35
        
        for i in range(len(images)):
            #mask_rle = rle_encode(masks[i])

            mask_rle_delete=contoursDelet(masks[i])
            mask_rle=rle_encode(mask_rle_delete)
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

    submit = pd.read_csv('./sample_submission.csv')
    submit['mask_rle'] = result

    submit.to_csv('./submit.csv', index=False)

In [ ]:
# # model 초기화
# model = UNet().to(device)

# # loss function과 optimizer 정의
# criterion = torch.nn.BCEWithLogitsLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # training loop
# for epoch in range(10):  # 10 에폭 동안 학습합니다.
#     model.train()
#     epoch_loss = 0
#     for images, masks in tqdm(dataloader):
#         images = images.float().to(device)
#         masks = masks.float().to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, masks.unsqueeze(1))
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#     print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

In [ ]:
# test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)

#         outputs = model(images)
#         masks = torch.sigmoid(outputs).cpu().numpy()
#         masks = np.squeeze(masks, axis=1)
#         masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35

#         for i in range(len(images)):
#             mask_rle = rle_encode(masks[i])
#             if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
#                 result.append(-1)
#             else:
#                 result.append(mask_rle)

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result

In [ ]:
# submit.to_csv('./submit.csv', index=False)

In [ ]:
# # create test dataloader (with preprocessing operation: to_tensor(...))
# test_dataset = BuildingsDataset(
#     TEST_DATA_DIR,
#     # augmentation=get_validation_augmentation(),
#     preprocessing=get_preprocessing(preprocessing_fn),
#     class_rgb_values=select_class_rgb_values,
# )

# test_dataloader = DataLoader(test_dataset)

# # test dataset for visualization (without preprocessing transformations)
# test_dataset_vis = BuildingsDataset(
#     TEST_DATA_DIR , TEST_LABLE_DIR ,
#     # augmentation=get_validation_augmentation(),
#     class_rgb_values=select_class_rgb_values,
# )

# # get a random test image/mask index
# random_idx = random.randint(0, len(test_dataset_vis)-1)
# print(random_idx)
# image, mask = test_dataset_vis[random_idx]

# visualize(
#     original_image = image,
#     ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask), select_class_rgb_values),
#     one_hot_encoded_mask = reverse_one_hot(mask)
# )
# # print(test_dataset[][0].shape)